In [1]:

#from dash import Dash, dcc, html
import os
import pandas as pd
pd.options.display.max_columns = None
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly
import jupyter_dash as jd
import dash
import graphs.plots as plots
from plotly.subplots import make_subplots

import ssl

ssl._create_default_https_context = ssl._create_unverified_context


#app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
#app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')

def data_prep(df):
    type_g = 'Renewal'
    df_heat = df.groupby(type_g).sum()#[['UW-Prämie','Abrech-Prämie','Ultimate-Prämie']]
    df_heat = df_heat[:len(df_heat.index.values)-2]
    df_heat['UW-SQ']=df_heat[['UW-Prämie','UW-Schaden']].apply(lambda x: x[0]/x[1], axis=1)
    df_heat['Abrech-SQ']=df_heat[['Abrech-Prämie','Abrech-Schaden']].apply(lambda x: x[0]/x[1], axis=1)
    df_heat['Ultimate-SQ']=df_heat[['Ultimate-Prämie','Ultimate-Schaden']].apply(lambda x: x[0]/x[1], axis=1)
    return df_heat


######################
app.layout = html.Div([
    html.H1('Rentabilitätsanalysis'),
    ######            
    dbc.Row([
#        dbc.Col(lg=1),
        dbc.Col([
            dcc.Dropdown(df['Description'].unique(),
                 'Canada', #multi=True,
                 id='country',),
            
#            style={"border":"1px black solid"},
            ]),# md=12, lg=5),
        
        dbc.Col([dcc.Graph(id='graph-p')]),# md=12, lg=5),
        dbc.Col([
            dcc.Dropdown(df.brch_text_kurz.unique(), 
                 "Windstorm", #multi=True, 
                 id='segment',),
            dcc.Graph(id='graph-all')
        ]),# md=12, lg=5),
    ],align='center'),
    dbc.Row([
        dbc.Col([ 
            
            ]),
        dbc.Col([ dcc.Graph(id='graph-ws')]),# md=12, lg=5),
        dbc.Col([ dcc.Graph(id='graph-loss-p')]),# md=12, lg=5),        
    ],align='center'),
])

@app.callback(
    dash.dependencies.Output('graph-all', 'figure'),
    [dash.dependencies.Input('segment', 'value')])
def update_figure(segment):
    ######## Parameter
    loc = segment
    var = 'brch_text_kurz'
    type_g = 'Renewal'
    ############

    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_loc = df[df[var]==loc]

    df_loc=df_loc.groupby(type_g).sum()
    df_loc=df_loc[:len(df_loc.index.values)-2]

    measure=[]
    for i in range(len(df_loc.index.values)-1):
        measure.append('relativ')
    measure.append('total')
    ###################################
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )
    fig.add_trace(
         go.Waterfall(
        name = "20", orientation = "v",
        measure = measure,
        x = df_loc.index.values,
        textposition = "outside",
        #text = ["+60", "+80", "-40", "-20", "Total"],
        y = df_loc['Abrech-Ergebnis'],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        ),
        row=2, col=1
    )
    ##################################
    df_loc = df[df[var]==loc]

    df_heat = df_loc.groupby(type_g).sum()#[['UW-Prämie','Abrech-Prämie','Ultimate-Prämie']]
    df_heat = df_heat[:len(df_heat.index.values)-2]
    fig.add_trace(
        go.Bar(
            x=df_heat.index.values,
            y=df_heat['UW-Prämie'],
            name="UW-Prämie",
            marker = {'color' : 'darkblue'}
        ),
        row=1, col=1
    )
    ###################
    fig.update_layout(
        height=300,
        width=600,
        showlegend=False,
        title_text="Prämienverlauf - "+loc,
        plot_bgcolor = "white"
    )
    return fig

@app.callback(
    dash.dependencies.Output('graph-bp', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_n=data_prep(df)
    return plots.get_bar(df_n)

@app.callback(
    dash.dependencies.Output('graph-p', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_n=data_prep(df[df['Description']==country])
    df_n['Description']=country
    return plots.get_bar(df_n)

@app.callback(
    dash.dependencies.Output('graph-ws', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    ######## Parameter
    loc = country
    var = 'Description'
    type_g = 'Renewal'
    ############

    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_loc = df[df[var]==loc]

    df_loc=df_loc.groupby(type_g).sum()
    df_loc=df_loc[:len(df_loc.index.values)-2]

    measure=[]
    for i in range(len(df_loc.index.values)-1):
        measure.append('relativ')
    measure.append('total')
    ###################################
    fig = make_subplots(
        rows=1, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )
    fig.add_trace(
         go.Waterfall(
        name = "20", orientation = "v",
        measure = measure,
        x = df_loc.index.values,
        textposition = "outside",
        #text = ["+60", "+80", "-40", "-20", "Total"],
        y = df_loc['Abrech-Ergebnis'],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        ),
        row=1, col=1
    )

    fig.update_layout(
        height=300,
        width=600,
        showlegend=False,
        title_text="Abwicklungsergebnis - "+loc,
        plot_bgcolor = "white"
    )
    return fig

@app.callback(
    dash.dependencies.Output('graph-wf', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    ######## Parameter
    loc = country
    type_g = 'Renewal'
    ############
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_loc = df
    df_loc=df_loc.groupby(type_g).sum()
    df_loc=df_loc[:len(df_loc.index.values)-2]

    measure=[]
    for i in range(len(df_loc.index.values)-1):
        measure.append('relativ')
    measure.append('total')
    ###################################
    fig = make_subplots(
        rows=1, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )
    fig.add_trace(
         go.Waterfall(
        name = "20", orientation = "v",
        measure = measure,
        x = df_loc.index.values,
        textposition = "outside",
        #text = ["+60", "+80", "-40", "-20", "Total"],
        y = df_loc['Abrech-Ergebnis'],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        ),
        row=1, col=1
    )

    fig.update_layout(
        height=300,
        width=600,
        showlegend=False,
        title_text="Abwicklungsergebnis",
        plot_bgcolor = "white"
    )
    return fig

@app.callback(
    dash.dependencies.Output('graph-loss-p', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_np=df[df['EboSchema']=='NP']
    df_np=df_np.groupby('Renewal').sum()[:len(df_np.index.values)-2]
    df_p=df[df['EboSchema']=='Prop']
    df_p=df_p.groupby('Renewal').sum()[:len(df_p.index.values)-2]
    #print(df_p)
    animals=df_p.index.values[:len(df_p.index.values)-2]

    loss_fig_npp = go.Figure(data=[
        go.Bar(name='NP', x=animals, y=df_np['Abrech-Schaden'], marker = {'color' : 'black'}),
        go.Bar(name='P', x=animals, y=df_p['Abrech-Schaden'], marker = {'color' : 'darkblue'})
    ])


    loss_fig_npp.update_layout(
        title_text="<b>Schadenaufteilung nach NP und P</b>", # Add figure title
        plot_bgcolor = "white", # background color
        barmode='stack' # Change the bar mode
    )
    # Set x-axis title
    loss_fig_npp.update_xaxes(title_text="<b>UW Year</b>")
    
    # legend
    loss_fig_npp.update_layout(showlegend=True)
    loss_fig_npp.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1
    ))
    loss_fig_npp.update_layout(
        height=300,
        width=600,
    )
    return loss_fig_npp

@app.callback(
    dash.dependencies.Output('graph-loss', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df=df.groupby('Renewal').sum()[:len(df.index.values)-2]
    animals=df.index.values[:len(df.index.values)-2]

    loss_fig = go.Figure(data=[
        go.Bar(x=animals, y=df['Abrech-Schaden'], marker = {'color' : 'darkblue'}),
    ])
    loss_fig.update_layout(
        title_text="<b>Schadenverlauf</b>", # Add figure title
        plot_bgcolor = "white", # background color
        #barmode='stack' # Change the bar mode
    )
    # Set x-axis title
    loss_fig.update_xaxes(title_text="<b>UW Year</b>")

    # legend
    loss_fig.update_layout(
        showlegend=False, 
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1,
            xanchor="right",
            x=1
    ))
    loss_fig.update_layout(
        height=300,
        width=600,
    )
    return loss_fig

if __name__ == '__main__':
    app.run_server(debug=False,port=8085)

C:\Users\Alk0h\AppData\Local\Temp\ipykernel_16964\1617818669.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Alk0h\AppData\Local\Temp\ipykernel_16964\1617818669.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
